In [ ]:
# import movie reviews and tag one sentence
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import movie_reviews
example_sentences = movie_reviews.sents()
example_sentence = example_sentences[0]
nltk.pos_tag(example_sentence)

In [ ]:
# parse one sentence
import spacy
text = "plot: two teen couples go to a church party, drink and then drive."
nlp = spacy.load("en_core_web_sm")
doc = nlp(text)
for token in doc:
    print (token.text, token.tag_, token.head.text, token.dep_)

In [ ]:
# compute count bag of words (Count BoW)
def document_features(document):
    features = {}
    for word in word_features:
        features[word] = 0
        for doc_word in document:
            if word == doc_word:
                features[word] += 1
                return features


In [ ]:
import string
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# consider only the most common words
max_tokens = 200

#this tokenizer will be used to tokenize the inputs
def tokenize(text):
    tokens = nltk.word_tokenize(text)
    return tokens

path = './movie_reviews/'
token_dict = {}

# look at all the files in the given path, there are 2,000 files
for dirpath, dirs, files in os.walk(path):
    for f in files:
        fname = os.path.join(dirpath, f)
        with open(fname) as review:
            text = review.read()
            token_dict[f] = text.lower().translate(str.maketrans('', '',string.punctuation))

#get a new tfIdf vectorizer            
tfIdfVectorizer = TfidfVectorizer(input = "content",
                                  use_idf = True,
                                  tokenizer = tokenize, 
                                  max_features = max_tokens, 
                                  stop_words = 'english')

# use the vectorizer to compute the tfIdf of the dataset
tfIdf = tfIdfVectorizer.fit_transform(token_dict.values())

# the feature names are the words (tokens) in the dataset
tfidf_tokens = tfIdfVectorizer.get_feature_names_out()

final_vectors = pd.DataFrame(
    data = tfIdf.toarray(), 
    columns = tfidf_tokens)

final_vectors

In [ ]:
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

#import the movie reviews
from nltk.corpus import movie_reviews

# make a list of movie review documents
documents = [(list(movie_reviews.words(fileid)))
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

# for this example, we'll just look at the first document, and 
# the first 50 words
data = documents[0]
values = array(data)
short_values = (values[:50])

# first encode words as integers
# every word in the vocabulary gets a unique number
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(short_values)

# look at the first 50 encodings
print(integer_encoded)

In [ ]:
# convert the integer encoding to onehot encoding
onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
onehot_encoded = onehot_encoder.fit_transform(integer_encoded)

print(onehot_encoded)

# invert the first vector so that we can see the original word it encodes
inverted = label_encoder.inverse_transform([argmax(onehot_encoded[0, :])])

print(inverted)

In [ ]:
# find similar words

import gensim
import nltk

from nltk.corpus import movie_reviews

from gensim.models import Word2Vec

# make a list of movie review documents
documents = [(list(movie_reviews.words(fileid)))
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

all_words = movie_reviews.words()
model = Word2Vec(documents, min_count=5)
model.wv.most_similar(positive = ['movie'],topn = 25)
